In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from utils import *
from tqdm import tqdm

dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
d = 3
m = 8
t = 200
n = 20000
snr_min = 0
snr_max = 0
lamda = 0.2
radius = 0.1
mc_range = 4
coherent = False

array = UCA(m, lamda)
array.build_array(radius=0.1)
array.build_array_manifold()
array.build_transform_matrices_from_array_manifold()

observations, angles = generate_data(n, t, d, snr_min, snr_max, array, mc_range, coherent)

In [3]:
class my_model(nn.Module): 
    
    def __init__(self, m: int, d: int, device=dev):
        
        super().__init__()

        self.m = m
        self.d = d
        self.device = device

        self.bn = nn.BatchNorm1d(2*m, device=device)
        self.rnn = nn.GRU(input_size=2*m, hidden_size=2*m, num_layers=1, batch_first=True, device=device)

        self.mlp = nn.Sequential(nn.Linear(in_features=2*m, out_features=2*m*m, device=device), nn.ReLU(),
                                 nn.Linear(in_features=2*m*m, out_features=2*m*m, device=device), nn.ReLU(),
                                 nn.Linear(in_features=2*m*m, out_features=d, device=device))

    def forward(self, x):
        
        x = torch.cat((torch.real(x), torch.imag(x)), dim=-1)
        x = self.bn(x.transpose(1, 2)).transpose(1, 2)
        _, x = self.rnn(x)
        y = self.mlp(x[-1])

        return y


In [4]:
nbEpoches = 200
lr = 1e-3
wd = 1e-9
batchSize = 256

x_train, x_valid, theta_train, theta_valid = train_test_split(observations, angles, test_size=0.2)
x_train, x_test, theta_train, theta_test = train_test_split(x_train, theta_train, test_size=0.2)

train_set = DATASET(x_train, theta_train)
valid_set = DATASET(x_valid, theta_valid)
test_set = DATASET(x_test, theta_test)

train_loader = DataLoader(train_set, batch_size=batchSize, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batchSize, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batchSize, shuffle=False)

In [5]:
model = my_model(m, d)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
Loss, Loss_theta, Val_theta = [], [], []
bestVal = 1000

for i in tqdm(range(nbEpoches)):
    # Train
    running_loss = 0.0
    for data in train_loader:
        X, theta_true = data[0].to(dev), data[1].to(dev)
        optimizer.zero_grad()
        theta_pred = model(X)
        loss = RMSPE(theta_pred, theta_true) 
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    Loss.append(running_loss/len(train_loader))

    # Validation 
    with torch.no_grad():
        running_loss = 0.0
        for data in valid_loader:
            X, theta_true = data[0].to(dev), data[1].to(dev)
            theta_pred = model(X)
            loss = RMSPE(theta_pred, theta_true)
            running_loss += loss.item()
        
        Val_theta.append(running_loss/len(test_loader))

        if Val_theta[i] < bestVal:
            bestVal = Val_theta[i]
            torch.save(model.state_dict(), 'model_3.pt')

    print("Iteration {}: Loss theta = {}".format(i, Loss[-1]))

  0%|          | 1/200 [00:02<08:27,  2.55s/it]

Iteration 0: Loss theta = 0.5749365657567977


  1%|          | 2/200 [00:04<07:55,  2.40s/it]

Iteration 1: Loss theta = 0.44494314074516295


  2%|▏         | 3/200 [00:07<07:44,  2.36s/it]

...

Iteration 197: Loss theta = 0.061845633313059804


100%|█████████▉| 199/200 [07:40<00:02,  2.37s/it]

Iteration 198: Loss theta = 0.06250588111579418


100%|██████████| 200/200 [07:43<00:00,  2.32s/it]

Iteration 199: Loss theta = 0.06117001324892044


In [6]:
model_test = my_model(m, d)
model_test.load_state_dict(torch.load('model_3.pt', weights_only=True))
running_loss = 0.0 

with torch.no_grad():
    for data in test_loader:
        X, theta_true= data[0].to(dev), data[1].to(dev)
        theta_pred = model_test(X)
        loss = RMSPE(theta_pred, theta_true)
        running_loss += loss.item()

Acc_theta = running_loss/len(test_loader)

print("RMSE DoA =", Acc_theta)

RMSE DoA = 0.06278936851483125
